In [10]:
from collections import namedtuple
import os
os.chdir("..")
os.getcwd()


'd:\\ineuron\\DL_project'

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path       #base untrained vgg16 archtecture model
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

@dataclass(frozen=True)
class CallbackPreparationConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
    
  

In [12]:
from cnnProject.constant import *
from cnnProject.utils import read_yaml,create_directories

In [13]:
class ConfigrationManger:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifact_root])

    def get_callback_perparation_config(self) -> CallbackPreparationConfig :
        config = self.config.callback_preparation

        model_checkpoint_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(config.tensorboard_root_log_dir),
            Path(model_checkpoint_dir)
        ])

        callback_prepare_config = CallbackPreparationConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )

        return callback_prepare_config
    
    
    def get_training_config(self) -> TrainingConfig :
        training = self.config.training
        prepare_base_model = self.config.base_model_preparation
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"PetImages")
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_epochs =  params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation = params.AUGMENTAION,
            params_image_size = params.IMAGE_SIZE
            )

        return training_config


In [14]:
#CallbackPrepareComponent
import os
import time
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

class CallbackPreparation:
    def __init__(self,config:CallbackPreparationConfig) -> None:
        self.config = config

    @property
    def _create_tb_callbacks(self):
        #good ideal to create a folder structure with timestamps
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")

        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )

        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)


    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True  #save only best weights aka loss was less
        )



    def get_tb_checkpoint_callback(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

   
    

In [15]:
#training Component
import os
import time
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

class Training:
    def __init__(self,config:TrainingConfig) -> None:
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        #will be doing augmentation 
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],  #only takes rows and columns, channels not aka index 0,1
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear",
            # interpolation is a process of determining the unknown values that lie in between the known data points.
            # ex find value between 1 and 2 on a line. 
        )


        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        #validation data
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset= "validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        #training data
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path,model: tf.keras.Model):
        model.save(path)


    #start training
    def train(self , callback_list):
        self.step_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.step_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator,
            callbacks = callback_list
        )

        self.save_model(
            path = self.config.trained_model_path,
            model = self.model
        )

    


In [16]:
try:
    config = ConfigrationManger()
    callback_preparation_config = config.get_callback_perparation_config()
    callback_prepare = CallbackPreparation(config=callback_preparation_config)
    callback_lst = callback_prepare.get_tb_checkpoint_callback()



    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()  #create a training and validation generator similar to train_test_split but not exactly  same 
    training.train(
        callback_list = callback_lst,
    )
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'Configrationfile\\config.yaml'

In [ ]:
#!pip install scipy

You should consider upgrading via the 'c:\python\python394\python.exe -m pip install --upgrade pip' command.


In [ ]:
#passing data as key value pair
def example(x,**kwargs):
    print(locals())


extra = dict(y=34,z=343)
example(x=2,**extra)

{'x': 2, 'kwargs': {'y': 34, 'z': 343}}


In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0
